# audio_converter by ANYONE U. I. N. K.

1: 下のセルを実行してpydubをインストール

In [ ]:
!pip install pydub

In [ ]:
2: 下のセルを実行してAudioConverterクラスをロード

In [ ]:
import os
import shutil
import glob
from pathlib import Path
from pydub import AudioSegment
from google.colab import files

class AudioConverter:
  def __init__(self, to_ext, unsafe_mode=False):
    self.to_ext = to_ext.lower().strip('.')
    self.ALLOW_INPUT_EXTS = {"aif", "aiff", "amr", "caf", "voc", "monkey", "ape", "mp2", "mp3", "mpc", "vqf", "adts", "aac", "ac3", "adx", "dts", "flac", "gsm", "mlp", "shn", "thd", "truehd", "wav", "3gp", "asf", "wmv", "avi", "bik", "dv", "swf", "flv", "fli", "flc", "gxf", "iff", "mkv", "mka", "mov", "mp4", "m4a", "mpg", "mpeg", "ps", "ts", "mxf", "nsv", "nut", "ogg", "ogv", "pmp", "pva", "rm", "vob"}

    if not unsafe_mode:
      if not self._validate_formats(self.to_ext):
        raise ValueError(f"Error: this converter does not support conversion to {self.to_ext}")

    self.base_dir = Path("/content/converter_workspace")
    self.input_dir = self.base_dir / "input"
    self.output_dir = self.base_dir / "output"
    self.uploaded_files_list = []

  def run(self, zip_name=""):
    self._setup_workspace()

    uploaded = files.upload()
    if not uploaded:
      print("the upload has been canceled")
      return True

    raw_files = self._process_uploaded_files(uploaded)

    valid_files = self._validate_files(raw_files)
    if not valid_files:
      print("Error: valid file does not exist")
      return False

    success_files, success_originals, failed_files = self._convert_all(valid_files)
    print(f"SUCCESS: {len(success_files)} files")
    print(f"FAILED: {len(failed_files)} files")
    if failed_files:
      print(f"failed files: {failed_files}")

    self._cleanup_originals(success_originals)

    if not success_files:
      print("Error: failed to convert all files")
      return False

    zip_path = self._create_zip(name=zip_name)
    self._download_file(zip_path)

    print("all processes have completed successfully")
    return True

  def _validate_formats(self, to_ext):
    if to_ext not in self.ALLOW_INPUT_EXTS:
        print(f"Error: .{to_ext} is not in standard supported list.")
        return False

    return True

  def _setup_workspace(self):
    print("starting the initializing workspace")

    if self.base_dir.exists():
      shutil.rmtree(self.base_dir)

    self.input_dir.mkdir(parents=True)
    self.output_dir.mkdir(parents=True)
    print(f"workspace initialized at {self.base_dir}")

  def _process_uploaded_files(self, uploaded_dict):
        moved_paths = []
        for filename in uploaded_dict.keys():
            src = Path(filename)
            dest = self.input_dir / src.name
            # overwrite and move if a file with the same name exists
            shutil.move(str(src), str(dest))
            moved_paths.append(dest)
        return moved_paths

  def _validate_files(self, file_list):
    valid_files = [f for f in file_list if f.suffix.lower().strip('.') != self.to_ext and f.suffix.lower().strip('.') in self.ALLOW_INPUT_EXTS]
    invalid_count = len(file_list) - len(valid_files)

    if invalid_count > 0:
      print(f"Warning: {invalid_count} invalid files have been excluded")

    return valid_files

  def _convert_all(self, file_paths):
    success_files = []
    success_originals = []
    failed_files = []

    print(f"starting the conversion of {len(file_paths)} files")

    for i, file_path in enumerate(file_paths, 1):
      current_ext = file_path.suffix.lower().strip('.')
      print(f"[{i}/{len(file_paths)}] converting: {file_path.name} ...", end="", flush=True)

      try:
        new_name = file_path.stem + f".{self.to_ext}"
        dest_path = self.output_dir / new_name

        audio = AudioSegment.from_file(file_path, format=current_ext)
        audio.export(dest_path, format=self.to_ext)

        success_files.append(new_name)
        success_originals.append(file_path)
        print(" success")
      except Exception as e:
        failed_files.append(file_path.name)
        print(f" failed: {e}")


    return success_files, success_originals, failed_files

  def _cleanup_originals(self, converted_paths):
    print("cleaning up successfully converted original files")
    for file_path in converted_paths:
      try:
        if file_path.exists():
          file_path.unlink()
      except Exception as e:
        print(f"Warning: could not delete {file_path.name}: {e}")

  def _create_zip(self, name="flac2wav"):
    # since shutil.make_archive automatically adds an extension, remove it to avoid ".zip.zip"
    zip_base_name = name.replace(".zip", "")

    zip_path_full = shutil.make_archive(
        base_name=str(self.base_dir / zip_base_name),
        format="zip",
        root_dir=str(self.output_dir)
    )

    print(f"created zip file: {os.path.basename(zip_path_full)}")
    return zip_path_full

  def _download_file(self, file_path):
    print(f"starting the download: {os.path.basename(file_path)}")
    files.download(file_path)

3: 下のセルのAudioConverterのなかの文字列を書き換えて変換したい拡張子にする

大体の音声ファイルには対応してるし、なんなら動画から音声ファイルにすることも可能

一応利用可能な拡張子はクラスにハードコーディングしているから気になるんなら見に行け

そこに登録してある拡張子以外でAudioConverterインスタンス化するとValueErrorになるようにしてある

もし変換したいファイルがValueErrorになったら追加するか
`AudioConverter("拡張子", unsafe_mode=True)`でインスタンス化しとけ

ファイルを選択ボタンを押したら複数選択できて、どんな拡張子でもまとめて変換する

あとはクソ長い処理を待ってダウンロード許可して終わり

In [ ]:
# @title AudioConverter { display-mode: "form" }

TARGET_FORMAT = "wav" # @param ["wav", "mp3", "m4a", "flac", "aac", "ogg", "opus", "mp4", "mov", "mkv", "avi", "wmv", "mpg", "mpeg", "ts", "flv", "webm", "aiff", "aif", "alac", "wma", "ac3", "dts", "caf", "amr", "ape", "thd", "truehd", "3gp", "asf", "m4v", "mka", "mxf", "vob", "rm", "swf", "ps", "nut", "voc", "mp2", "mpc", "adx", "adts", "shn", "mlp", "gsm", "vqf", "bik", "dv", "fli", "flc", "gxf", "iff", "nsv", "pmp", "pva"]
UNSAFE_MODE = False # @param {type:"boolean"}
ZIP_NAME = "" # @param {type:"string"}

# ここで初期化
converter = AudioConverter(to_ext=TARGET_FORMAT, unsafe_mode=UNSAFE_MODE)
converter.run(zip_name=ZIP_NAME)